In [1]:
import os
import re
import codecs
import json
from datetime import datetime

In [2]:
def rename_data_file(**kwargs):
    """
    Funcao que recebe o nome do arquivo original e o renomeia.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: '20110101_AC.txt'
        file_ext(str): Tipo de extensao a ser adicionado ao nome do arquivo.
            Ex.: '.csv'
    
    Returns:
        string: Nome do arquivo modificado.
            Ex.: '20110101_CNPJ.csv'
    """
    # Expressao Regular para:
    # Identificar a data de referencia no nome do arquivo
    date_reference_re = re.compile(r'^[0-9]+')
    # Identificar o assunto no nome do arquivo
    file_subject_re = re.compile(r'_([\w]+)_([0-9]+)\.')
    
    # Identifica a data de referencia do arquivo
    date_reference = re.search(date_reference_re, kwargs['data_file']).group()
    # Identifica o assunto do arquivo
    subject = re.search(file_subject_re, kwargs['data_file']).group(1)
    
    # Define o novo nome do arquivo
    renamed_file = date_reference + '_' + subject + kwargs['file_ext']
    
    return renamed_file

In [3]:
def json_to_csv(**kwargs):
    """
    Le o arquivo JSON de entrada e converte para um arquivo de saida CSV.
    
    Args:
        in_data_path(str): Caminho completo do arquivo de entrada.
        out_data_dir(str): Caminho da pasta para armazenamento do arquivo de saida.
        data_file(str): Nome do arquivo de entrada.
        delimiter(str): Delimitador de campos utilizado no arquivo de saida
        file_ext(str): Extensao do arquivo de saida
        encoding(str): Codificacao do arquivo de saida
    """
    
    # Nome do arquivo de saida
    kwargs['out_data_file'] = rename_data_file(**kwargs)
    # Caminho completo do arquivo de saida
    kwargs['out_data_path'] = os.path.join(kwargs['out_data_dir'], kwargs['out_data_file'])
    
    # Verifica se o arquivo de saida ja existe
    # Se existir, armazena a variavel para abrir o arquivo de saida
    # Caso contrario, armazena a variavel para criar o arquivo de saida
    if os.path.exists(kwargs['out_data_path']):
        kwargs['open_mode'] = 'a'
    else:
        kwargs['open_mode'] = 'w'
    
    # Log: Mensagem de inicio da conversao
    print(str(datetime.now()) + ': Conversao do arquivo ' + kwargs['data_file'] + ' iniciada.')
    
    # Le o arquivo de entrada
    with codecs.open(kwargs['in_data_path'], 'r', encoding = kwargs['encoding']) as reader:
        
        # Prepara o arquivo json
        data = json.load(reader)
        
        # Define o header para o arquivo de saida separado por tabulacao
        kwargs['header'] = 'id_siafi_orgao\tds_siafi_orgao\tds_siafi_orgao_formatado\n'
              
        # Prepara o arquivo de saida
        with codecs.open(kwargs['out_data_path'], kwargs['open_mode'], encoding = kwargs['encoding']) as writer:
            
            # Caso o modo de abertura seja para criacao
            # escreve o cabecalho no arquivo de saida
            if kwargs['open_mode'] == 'w' :
                writer.write(kwargs['header'])
            
            # Para cada objeto no arquivo de entrada
            for d in data:
            
                # Escreve o objeto no formato tabular no arquivo de saida
                    writer.write(
                                                d['codigo']                   # id_siafi_orgao
                        + kwargs['delimiter'] + d['descricao']                # ds_siafi_orgao
                        + kwargs['delimiter'] + d['codigoDescricaoFormatado'] # ds_siafi_orgao_formatado 
                        + '\n'
                    )
                    
    # Log: Resultado da verificacao dos registros
    print(str(datetime.now()) + ': Conversao do arquivo JSON para CSV finalizada.')

In [4]:
def organize_files():
    """
    Para cada arquivo de origem, executa a reorganizacao do arquivo de JSON para CSV.        
    """
    
    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Diretorio de armazenamento dos arquivos originais
    kwargs['in_data_dir'] = '..\\data\\02-cleaned'
    
    # Diretorio de armazenamento dos arquivos tratados
    kwargs['out_data_dir'] = '..\\data\\03-organized'
    
    # Lista dos arquivos originais 
    kwargs['data_files'] = os.listdir(kwargs['in_data_dir'])
    
    # Parametros de organizacao dos arquivos delimitados
    kwargs['delimiter'] = '\t'
    kwargs['file_ext'] = '.csv'
    kwargs['encoding'] = 'utf-8'
    
    # Log: Mensagem de inicio do processo 
    print(str(datetime.now()) + ': Organizacao dos arquivos iniciada.')
    
    # Remove qualquer arquivo de saida que tenha sido anteriormente criado
    old_files = [os.path.join(kwargs['out_data_dir'],f) for f in os.listdir(kwargs['out_data_dir'])]
    for f in old_files:
        os.remove(f) 
    
    # Para cada arquivo na lista de arquivos originais
    for file in kwargs['data_files']:
        
        # Define o caminho completo de acesso e escrita dos arquivos
        kwargs['data_file'] = file
        kwargs['in_data_path'] = os.path.join(kwargs['in_data_dir'], kwargs['data_file'])
        kwargs['out_data_path'] = os.path.join(kwargs['out_data_dir'], kwargs['data_file'])
        
        # Corrige registros com menos colunas que o esperado
        json_to_csv(**kwargs)
      
    # Log: Mensagem de finalizacao do processo de limpeza
    print(str(datetime.now()) + ': Organizacao dos arquivos finalizada.')

In [5]:
def main():
    """
    Organiza os dados de Orgaos cadastrados no SIAFI.
    Origem: Portal da Transparencia
    """
    
    ### Organiza os arquivos
    organize_files()

if __name__ == '__main__':
    main()

2019-09-10 13:22:29.337799: Organizacao dos arquivos iniciada.
2019-09-10 13:22:29.341786: Conversao do arquivo 20190906_orgaossiafi_1.json iniciada.
2019-09-10 13:22:29.348768: Conversao do arquivo JSON para CSV finalizada.
2019-09-10 13:22:29.349772: Conversao do arquivo 20190906_orgaossiafi_10.json iniciada.
2019-09-10 13:22:29.352756: Conversao do arquivo JSON para CSV finalizada.
2019-09-10 13:22:29.352756: Conversao do arquivo 20190906_orgaossiafi_11.json iniciada.
2019-09-10 13:22:29.354752: Conversao do arquivo JSON para CSV finalizada.
2019-09-10 13:22:29.355752: Conversao do arquivo 20190906_orgaossiafi_12.json iniciada.
2019-09-10 13:22:29.357778: Conversao do arquivo JSON para CSV finalizada.
2019-09-10 13:22:29.358741: Conversao do arquivo 20190906_orgaossiafi_13.json iniciada.
2019-09-10 13:22:29.360781: Conversao do arquivo JSON para CSV finalizada.
2019-09-10 13:22:29.361731: Conversao do arquivo 20190906_orgaossiafi_14.json iniciada.
2019-09-10 13:22:29.365722: Convers